In [2]:
import nibabel as nb

In [3]:
input1='../sourcedata/data/HCP_7T_movie_FIX/brain/HCP_7T_movie_FIX/100610/MNINonLinear/Results/tfMRI_MOVIE1_7T_AP/tfMRI_MOVIE1_7T_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
tr=1
img = nb.load(input1)
X = img.get_fdata()
X.shape

In [17]:
import argparse
from pathlib import Path
import numpy as np
#import soundfile as sf
import librosa

ModuleNotFoundError: No module named 'librosa'

## 